In [1]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# conda activate PythonAdv

In [3]:
# Import data
df = pd.read_csv('news/news.csv')
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
# df.columns ['id', 'title', 'text', 'label']

In [5]:
test_df = df.rename(columns={"Unnamed: 0": "id"})
test_df.head()

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [24]:
# test_df = test_df.set_index("id")
# test_df.head()

In [30]:
# test_df.head()

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [8]:
# test_df["title"].to_csv("new_test_data_title.csv", index=True)
# test_df["text"].to_csv("new_test_data_text.csv", index=True)
# test_df["label"].to_csv("new_test_data_label.csv", index=True)

In [9]:
test_df.to_json("new_test_data.json", index=True, orient="records")

In [10]:
test_df = pd.read_json("new_test_data.json")

In [11]:
test_df.dtypes

index     int64
id        int64
title    object
text     object
label    object
dtype: object

In [12]:
labels=df.label
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [15]:
# !pip install nltk

  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434676 sha256=e49688cf4192f26c792d55f847321252f8024c7c2ffe83081374a181d781fe4c
  Stored in directory: c:\users\alext\appdata\local\pip\cache\wheels\de\5e\42\64abaeca668161c3e2cecc24f864a8fc421e3d07a104fc8a51
Successfully built nltk


In [57]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [58]:
test_df.head()

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [66]:
# Label-encode
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(test_df["label"])
test_df['encoded_labels'] = label_encoder.transform(test_df["label"])

In [67]:
# Tokenize text and remove stop words
stop = stopwords.words('english')
test_df['token_text'] = test_df["text"].apply(word_tokenize)
test_df['clean_token_text'] = test_df["token_text"].apply(lambda x: [item for item in x if item not in stop])
test_df.head()

,id,title,text,label,token_text,clean_token_text,encoded_labels
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,"[Daniel, Greenfield, ,, a, Shillman, Journalis...","[Daniel, Greenfield, ,, Shillman, Journalism, ...",0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,"[Google, Pinterest, Digg, Linkedin, Reddit, St...","[Google, Pinterest, Digg, Linkedin, Reddit, St...",0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,"[U.S., Secretary, of, State, John, F., Kerry, ...","[U.S., Secretary, State, John, F., Kerry, said...",1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,"[—, Kaydee, King, (, @, KaydeeKing, ), Novembe...","[—, Kaydee, King, (, @, KaydeeKing, ), Novembe...",0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,"[It, 's, primary, day, in, New, York, and, fro...","[It, 's, primary, day, New, York, front-runner...",1


In [ ]:
# Split dataset to training and testing data

# X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.2, random_state=7)

In [ ]:
# # Initialize a TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# # Fit and transform train set, transform test set
# tfidf_train = tfidf_vectorizer.fit_transform(X_train) 
# tfidf_test=tfidf_vectorizer.transform(X_test)

In [41]:
# # Reformat data
# data = test_df.values
# X = data[:, 0:3]
# y = data[:, 3]

In [46]:
# # Label-encode
# from sklearn.preprocessing import LabelEncoder

# label_encoder = LabelEncoder()
# label_encoder.fit(y)
# encoded_y = label_encoder.transform(y)

In [37]:
# encoded_y, y

(array([0, 0, 1, ..., 0, 1, 1]),
 array(['FAKE', 'FAKE', 'REAL', ..., 'FAKE', 'REAL', 'REAL'], dtype=object))

In [47]:
# Split dataset to training and testing data

# X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.2, random_state=7)

In [53]:
# X_train

array([[7836,
        'Shock Claim: Nibiru Will Hit In 2017- World Governments Are Prepared',
        'The head of a leading survivalist group has made several shocking allegations about the United States government building secret, cavernous underground bunkers which are earmarked for use by the global elite in the case of an imminent apocalyptic event.\n\n\nA vast underground bunker under Denver Airport is ready for use by the global elite \n\nRobert Vicino, head of the survival firm Vivos has claimed that officials from the United States have been building ‘deep underground shelters’ underneath the city of Denver for that past three decades. He claims that sources have deduced that these survival shelters have been a work in progress since 1983. The largest has space for 10,000 people who are members of the global elite. However, the general population has not be considered in these plans. "They do not have a plan for you and me, but they have a plan for themselves, ” said Vicino.\n

In [49]:
# One-hot encode the labels
# from tensorflow.keras.utils import to_categorical

# y_train_categorical = to_categorical(y_train)
# y_test_categorical = to_categorical(y_test)

In [52]:
# # Initialize a TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# # Fit and transform train set, transform test set
# tfidf_train = tfidf_vectorizer.fit_transform(X_train) 
# tfidf_test=tfidf_vectorizer.transform(X_test)


AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [ ]:
# Initialize a PassiveAggressiveClassifier
# pac=PassiveAggressiveClassifier(max_iter=50)
# pac.fit(tfidf_train,y_train)
# # Predict on the test set and calculate accuracy
# y_pred=pac.predict(tfidf_test)
# score=accuracy_score(y_test,y_pred)
# print(f'Accuracy: {round(score*100,2)}%')

In [ ]:
# Build confusion matrix
# confusion_matrix(y_test,y_pred, labels=['FAKE','REAL'])